In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, UpSampling1D
from keras.layers import ReLU, Dropout, Activation, AveragePooling1D, Flatten, Reshape, Conv1D, MaxPooling1D, Embedding, GlobalAveragePooling1D, Conv2D, BatchNormalization, LeakyReLU, Conv2DTranspose, Concatenate, LSTM, Lambda
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler, TensorBoard
from keras import optimizers, metrics
from keras.utils import plot_model
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Sequential, save_model, load_model
import tensorflow as tf
import csv
from sklearn.utils import shuffle
from tensorflow.keras import layers
from keras.layers import Input, Dense, BatchNormalization, Reshape, Conv2D, add, LeakyReLU

In [2]:
input = Input(batch_shape = (None, 1024, 1))

In [3]:
def convblock(filter_size, kernel, inputs):
    c_1 = Conv1D(filter_size, kernel, dilation_rate = 2, activation = 'relu', padding = 'same')(inputs)
    c_2 = Conv1D(filter_size, kernel, dilation_rate = 2, padding = 'same')(c_1)

    if c_2.shape[-1] != inputs.shape[-1]:
        c_2=layers.Dense(inputs.shape[-1])(c_2)
        
    c_2 = layers.Add()([c_2, inputs])
    c_2 = layers.ReLU()(c_2)

    return c_2

In [4]:
c1 = Conv1D(64, 7, strides = 2, activation = 'relu', padding = 'same')(input)
c1 = MaxPooling1D(3, 2)(c1)

c2a = convblock(64, 3, c1)
c2b = convblock(64, 4, c1)

c3a = convblock(128, 3, c2a)
c3b = convblock(128, 4, c2b)

c4a = convblock(256, 3, c3a)
c4b = convblock(256, 4, c3b)

c5a = convblock(256, 3, c4a)
c5b = convblock(256, 4, c4b)

c_a = layers.AveragePooling1D()(c5a)
c_a = layers.Dropout(0.2)(c_a)

c_b = layers.AveragePooling1D()(c5b)
c_b = layers.Dropout(0.2)(c_b)

final = layers.Add()([c_a, c_b])
final = layers.Flatten()(final)

outputs = layers.Dense(10, activation = 'softmax')(final)

model = tf.keras.Model(input, final)
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1024, 1)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 512, 64)      512         ['input_1[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 255, 64)      0           ['conv1d[0][0]']                 
                                                                                                  
 conv1d_1 (Conv1D)              (None, 255, 64)      12352       ['max_pooling1d[0][0]']          
                                                                                              